In [8]:
__author__ = 'Evgeny BAZAROV'

FOLDER = "/home/evgeny/kaggle/input/"

import jellyfish
import pandas as pd

from pyxdameraulevenshtein import normalized_damerau_levenshtein_distance

import re

pd.set_option("display.max_columns", 99)

In [18]:
# TRAIN SET
pd_data_part1 = pd.read_hdf(FOLDER + "train_merged-part1.h")
pd_data_part2 = pd.read_hdf(FOLDER + "train_merged-part2.h")
pd_data = pd_data_part1.append(pd_data_part2)

pd_data.description_1.fillna("", inplace=True)
pd_data.description_2.fillna("", inplace=True)

pd_data.title_1.fillna("", inplace=True)
pd_data.title_2.fillna("", inplace=True)

del pd_data_part1
del pd_data_part2

print("DONE")

DONE


In [19]:
pd_data.columns

Index(['itemID_1', 'itemID_2', 'isDuplicate', 'generationMethod',
       'categoryID_1', 'title_1', 'description_1', 'images_array_1',
       'attrsJSON_1', 'price_1', 'locationID_1', 'metroID_1', 'lat_1', 'lon_1',
       'parentCategoryID_1', 'regionID_1', 'categoryID_2', 'title_2',
       'description_2', 'images_array_2', 'attrsJSON_2', 'price_2',
       'locationID_2', 'metroID_2', 'lat_2', 'lon_2', 'parentCategoryID_2',
       'regionID_2'],
      dtype='object')

In [20]:
def get_features(col1, col2, function):
    new_feat = []
    ctx = 0
    for x, y in zip(col1, col2):
        ctx += 1
        new_feat.append(function(x, y))
        if ctx % 100000 == 0:
            print(ctx)
    pd_feat = pd.DataFrame(new_feat)
    return pd_feat

#cols = [('title_1', 'title_2'), ('description_1', 'description_2')]
cols = [('attrsJSON_1', 'attrsJSON_2')]

for col1, col2 in cols:
    print("Selecting: {} and {}".format(col1, col2))
    colname = col1.split("_")[0]
    col_1 = pd_data['title_1'].values
    col_2 = pd_data['title_2'].values

    print("jaro winkler")
    pd_feat = get_features(col_1, col_2, jellyfish.jaro_winkler)
    pd_feat.to_hdf(FOLDER + "feat_train_{}_jarowinkler.h".format(colname), 'w')

    print("normalized demarau levenstein")
    pd_feat = get_features(col_1, col_2, normalized_damerau_levenshtein_distance)
    pd_feat.to_hdf(FOLDER + "feat_train_{}_demarauleven.h".format(colname), 'w')

Selecting: attrsJSON_1 and attrsJSON_2
jaro winkler
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
normalized demarau levenstein
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000


In [6]:
pd_data['title_1_num'] = pd_data.title_1.apply(lambda x: re.sub('[^0-9]','', x))
pd_data['title_2_num'] = pd_data.title_2.apply(lambda x: re.sub('[^0-9]','', x))

In [7]:
pd_data['title_num_same'] = 1 * (pd_data['title_1_num'] == pd_data['title_2_num'])

In [13]:
pd_feat = pd_data['title_num_same']
pd_feat.to_hdf(FOLDER + "feat_train_title_num_same.h", 'w')

In [21]:
# TEST SET

pd_data = pd.read_hdf(FOLDER + "test_merged.h")

pd_data.description_1.fillna("", inplace=True)
pd_data.description_2.fillna("", inplace=True)

pd_data.title_1.fillna("", inplace=True)
pd_data.title_2.fillna("", inplace=True)

#cols = [('title_1', 'title_2'), ('description_1', 'description_2')]
cols = [('attrsJSON_1', 'attrsJSON_2')]


In [22]:
for col1, col2 in cols:
    print("Selecting: {} and {}".format(col1, col2))
    colname = col1.split("_")[0]
    col_1 = pd_data['title_1'].values
    col_2 = pd_data['title_2'].values

    print("jaro winkler")
    pd_feat = get_features(col_1, col_2, jellyfish.jaro_winkler)
    pd_feat.to_hdf(FOLDER + "feat_test_{}_jarowinkler.h".format(colname), 'w')

    print("normalized demarau levenstein")
    pd_feat = get_features(col_1, col_2, normalized_damerau_levenshtein_distance)
    pd_feat.to_hdf(FOLDER + "feat_test_{}_demarauleven.h".format(colname), 'w')

Selecting: attrsJSON_1 and attrsJSON_2
jaro winkler
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
normalized demarau levenstein
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000


In [17]:
pd_data['title_1_num'] = pd_data.title_1.apply(lambda x: re.sub('[^0-9]','', x))
pd_data['title_2_num'] = pd_data.title_2.apply(lambda x: re.sub('[^0-9]','', x))
pd_data['title_num_same'] = 1 * (pd_data['title_1_num'] == pd_data['title_2_num'])
pd_feat = pd_data['title_num_same']
pd_feat.to_hdf(FOLDER + "feat_test_title_num_same.h", 'w')